In [53]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

file_path = '/home/carolus/Documents/school/green_ia/data/02_openfoodfacts_sample.jsonl'

df = pd.read_json(file_path, lines=True)

In [54]:
df.head(60)

,groups,packaging,name,ecoscore_groups,ecoscore_note,code,countries,ingredients,categories,labels_note
0,Fat and sauces,None,"Nalley, sandwich creations, shredded chicken in taco sauce",z,999,41321316159,united states,"chicken, poultry, sauce, chicken-meat, water, salt, modified-starch, starch, e339, spice, condiment, modified-corn-starch, corn-starch, maltodextrin, sugar, added-sugar, disaccharide, tomato, vegetable, fruit-vegetable, onion, root-vegetable, onion-family-vegetable, dextrose, monosaccharide, glucose, garlic, green-bell-pepper, bell-pepper, natural-flavouring, flavouring, e415, including-chipotle-chill-pepper, natural-smoke-flavouring","canned-foods, condiments, meats-and-their-products, meats, sauces, canned-meats, groceries",0
1,None,None,"Shirakiku, pickled plums, honey",z,999,74410120160,united states,"plum, fruit, prunus-species-fruit, salt, e621, honey, added-sugar, cooking-wine, e260, e202, glucose, monosaccharide, e1510, rice, rice-malt, e330, e363, e640i, e640, e631, e627, water, preservative",salted-snacks,0
2,Milk and dairy products,None,"Golden milk lassi, golden milk",b,67,793573014764,united states,"pasteurised-whole-milk, dairy, milk, pasteurised-milk, whole-milk, cane-sugar, added-sugar, disaccharide, sugar, ginger, condiment, spice, tumeric, indian-spices, microbial-culture, ferment, probiotic-strains, bifidobacterium-animalis-bb-12, lactobacillus-acidophilus-la-5, l, paracasei, propionibacterium-freudenreichii, streptococcus-thermophilus, lactic-ferments","dairies, fermented-foods, fermented-milk-products, desserts, dairy-desserts, fermented-dairy-desserts, yogurts",0
3,Fat and sauces,"Verre, Bocal, Bocal en verre, Couvercle en métal",Pesto Orange,c,49,3302768000436,france,"carrot, vegetable, root-vegetable, taproot-vegetable, colza-oil, oil-and-fat, vegetable-oil-and-fat, rapeseed-oil, sunflower-oil, vegetable-oil, water, parmigiano-reggiano, dairy, cheese, cream, pecan-nut, nut, tree-nut, basil, herb, salt, extra-virgin-olive-oil, olive-oil, virgin-olive-oil, garlic, onion-family-vegetable, citrus-fibre, fiber, vegetable-fiber, e270, natural-paprika-extract, e160c, pepper, seed","condiments, sauces, pestos, groceries",2
4,None,None,Pasta sauce,z,999,3662444002285,france,None,None,0
5,None,None,Dely - 4 Brussels waffles Frozen,z,999,5412279841232,switzerland,"water, wheat-flour, cereal, flour, wheat, cereal-flour, colza-oil, oil-and-fat, vegetable-oil-and-fat, rapeseed-oil, margarine, egg-yolk, egg, sugar, added-sugar, disaccharide, raising-agent, soya-flour, vegetable, legume, pulse, soya, soya-bean, salt, vanilla-flavouring, flavouring, emulsifier, acid, colour, e450i, e450, e500ii, e500, e503ii, e503, e471, e330, e160ai, e160a",None,0
6,Beverages,None,Sok z pokrzywy,e,4,5901812776150,poland,None,"plant-based-foods-and-beverages, beverages, plant-based-beverages, fruit-based-beverages, juices-and-nectars, fruit-juices, squeezed-juices, nettle-juices",0
7,None,None,CLA 1000,z,999,3770001283128,france,None,None,0
8,None,None,Salade du pecheur,z,999,262046027940,france,None,None,0
9,Cereals and potatoes,None,salted rice cakes with sea salt,c,43,5099839281709,france,None,"plant-based-foods-and-beverages, plant-based-foods, cereals-and-potatoes, cereals-and-their-products, puffed-cereal-cakes, puffed-rice-cakes",0


In [55]:
valeurs_uniques = df['packaging'].unique()
print(valeurs_uniques)

[None 'Verre, Bocal, Bocal en verre, Couvercle en métal' 'Brique' 'packed'
 'Glas, Flasche' 'Pp-tub, Tub' 'Plástico' 'mixed plastic film-packet']


In [56]:
df['countries'].tail(50)

10            france
11            france
12             spain
13            france
14             spain
15             spain
16             spain
17          thailand
18     united states
19     united states
20     united states
21            france
22       deutschland
23            france
24             spain
25            france
26            canada
27             spain
28    united kingdom
29            canada
30            france
31             spain
32             spain
33    united kingdom
34             italy
35             italy
36     united states
37     united states
38             italy
39     united states
40             italy
41             spain
42           germany
43             spain
44         australia
45     united states
46             italy
47    united kingdom
48            canada
49     united states
50    united kingdom
51             italy
52             spain
53            france
54             italy
55           ireland
56           germany
57     united

In [57]:
nan_counts = df.isna().sum()

# Compter les listes vides
empty_list_counts = df.map(lambda x: x == []).sum()

# Afficher les résultats
print("nombre de NaN ou None par colonne :")
print(nan_counts)

nombre de NaN ou None par colonne :
groups             41
packaging          53
name                0
ecoscore_groups     0
ecoscore_note       0
code                0
countries           0
ingredients        45
categories         38
labels_note         0
dtype: int64


In [58]:
df['labels_note'].head(50)

0     0
1     0
2     0
3     2
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    2
12    0
13    0
14    0
15    3
16    0
17    1
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    2
27    0
28    1
29    0
30    0
31    0
32    0
33    0
34    0
35    1
36    0
37    0
38    0
39    0
40    0
41    1
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
Name: labels_note, dtype: int64

In [59]:
df['ingredients'].head(50)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         chicken, poultry, sauce, chicken-meat, water, salt, modified-starch, starch, e339, spice, condiment, modified-corn-starch, corn-starch, maltodextrin, sugar, added-sugar, disaccharide, tomato, vegetable, fruit-vegetable, onion, root-vegetable, onion-family-vegetable, dextrose, monosaccharide, glucose, garlic, green-bell-pepper, bell-